In [1]:
#python のバージョンの表示
import sys
print(sys.version)

3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [2]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:24<00:00, 202.40it/s]

count     win    lose    draw
o        2931    1444     625
x        1414    2918     668
total    4345    4362    1293

ratio     win    lose    draw
o       58.6%   28.9%   12.5%
x       28.3%   58.4%   13.4%
total   43.5%   43.6%   12.9%



[('count',
  [{'win': 2931, 'lose': 1444, 'draw': 625},
   {'win': 1414, 'lose': 2918, 'draw': 668},
   {'win': 4345, 'lose': 4362, 'draw': 1293}],
  '7d'),
 ('ratio',
  [{'win': 0.5862, 'lose': 0.2888, 'draw': 0.125},
   {'win': 0.2828, 'lose': 0.5836, 'draw': 0.1336},
   {'win': 0.4345, 'lose': 0.4362, 'draw': 0.1293}],
  '7.1%')]

In [3]:
from marubatsu import Marubatsu

def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 〇 の勝利の判定
    if self.is_winner(Marubatsu.CIRCLE):
        return Marubatsu.CIRCLE
    # × の勝利の判定
    elif self.is_winner(Marubatsu.CROSS):
        return Marubatsu.CROSS   
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [4]:
from test import test_judge

test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [5]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:09<00:00, 512.89it/s]

count     win    lose    draw
o        2865    1487     648
x        1344    3028     628
total    4209    4515    1276

ratio     win    lose    draw
o       57.3%   29.7%   13.0%
x       26.9%   60.6%   12.6%
total   42.1%   45.1%   12.8%



[('count',
  [{'win': 2865, 'lose': 1487, 'draw': 648},
   {'win': 1344, 'lose': 3028, 'draw': 628},
   {'win': 4209, 'lose': 4515, 'draw': 1276}],
  '7d'),
 ('ratio',
  [{'win': 0.573, 'lose': 0.2974, 'draw': 0.1296},
   {'win': 0.2688, 'lose': 0.6056, 'draw': 0.1256},
   {'win': 0.4209, 'lose': 0.4515, 'draw': 0.1276}],
  '7.1%')]

In [6]:
def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [7]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [8]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:07<00:00, 674.19it/s]

count     win    lose    draw
o        2873    1474     653
x        1424    2920     656
total    4297    4394    1309

ratio     win    lose    draw
o       57.5%   29.5%   13.1%
x       28.5%   58.4%   13.1%
total   43.0%   43.9%   13.1%



[('count',
  [{'win': 2873, 'lose': 1474, 'draw': 653},
   {'win': 1424, 'lose': 2920, 'draw': 656},
   {'win': 4297, 'lose': 4394, 'draw': 1309}],
  '7d'),
 ('ratio',
  [{'win': 0.5746, 'lose': 0.2948, 'draw': 0.1306},
   {'win': 0.2848, 'lose': 0.584, 'draw': 0.1312},
   {'win': 0.4297, 'lose': 0.4394, 'draw': 0.1309}],
  '7.1%')]

In [9]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [10]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [11]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [12]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:05<00:00, 959.18it/s] 

count     win    lose    draw
o        2868    1465     667
x        1412    2960     628
total    4280    4425    1295

ratio     win    lose    draw
o       57.4%   29.3%   13.3%
x       28.2%   59.2%   12.6%
total   42.8%   44.2%   13.0%



[('count',
  [{'win': 2868, 'lose': 1465, 'draw': 667},
   {'win': 1412, 'lose': 2960, 'draw': 628},
   {'win': 4280, 'lose': 4425, 'draw': 1295}],
  '7d'),
 ('ratio',
  [{'win': 0.5736, 'lose': 0.293, 'draw': 0.1334},
   {'win': 0.2824, 'lose': 0.592, 'draw': 0.1256},
   {'win': 0.428, 'lose': 0.4425, 'draw': 0.1295}],
  '7.1%')]

In [13]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    self.rowcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.colcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.diacount = {
        Marubatsu.CIRCLE: [0] * 2,
        Marubatsu.CROSS: [0] * 2,
    }
    
Marubatsu.restart = restart

In [14]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.colcount[self.last_turn][x] += 1
        self.rowcount[self.last_turn][y] += 1
        if x == y:
            self.diacount[self.last_turn][0] += 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] += 1        
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [15]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        self.colcount[self.last_turn][x] -= 1
        self.rowcount[self.last_turn][y] -= 1
        if x == y:
            self.diacount[self.last_turn][0] -= 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] -= 1           
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        
        self.last_move = self.records[-1]  
        
Marubatsu.unmove = unmove

In [16]:
def is_winner(self, player):
    x, y = self.last_move
    if self.rowcount[player][y] == self.BOARD_SIZE or \
       self.colcount[player][x] == self.BOARD_SIZE:
        return True
    # 左上から右下方向の判定
    if x == y and self.diacount[player][0] == self.BOARD_SIZE:
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.diacount[player][1] == self.BOARD_SIZE:
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [17]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [18]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:05<00:00, 954.89it/s] 

count     win    lose    draw
o        2947    1417     636
x        1471    2926     603
total    4418    4343    1239

ratio     win    lose    draw
o       58.9%   28.3%   12.7%
x       29.4%   58.5%   12.1%
total   44.2%   43.4%   12.4%



[('count',
  [{'win': 2947, 'lose': 1417, 'draw': 636},
   {'win': 1471, 'lose': 2926, 'draw': 603},
   {'win': 4418, 'lose': 4343, 'draw': 1239}],
  '7d'),
 ('ratio',
  [{'win': 0.5894, 'lose': 0.2834, 'draw': 0.1272},
   {'win': 0.2942, 'lose': 0.5852, 'draw': 0.1206},
   {'win': 0.4418, 'lose': 0.4343, 'draw': 0.1239}],
  '7.1%')]

In [19]:
def judge(self):
    if self.move_count == 0:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [20]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [21]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:05<00:00, 953.70it/s] 

count     win    lose    draw
o        2953    1423     624
x        1462    2933     605
total    4415    4356    1229

ratio     win    lose    draw
o       59.1%   28.5%   12.5%
x       29.2%   58.7%   12.1%
total   44.1%   43.6%   12.3%



[('count',
  [{'win': 2953, 'lose': 1423, 'draw': 624},
   {'win': 1462, 'lose': 2933, 'draw': 605},
   {'win': 4415, 'lose': 4356, 'draw': 1229}],
  '7d'),
 ('ratio',
  [{'win': 0.5906, 'lose': 0.2846, 'draw': 0.1248},
   {'win': 0.2924, 'lose': 0.5866, 'draw': 0.121},
   {'win': 0.4415, 'lose': 0.4356, 'draw': 0.1229}],
  '7.1%')]

In [22]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [23]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [24]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [25]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]  

Marubatsu.unmove = unmove

In [26]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [27]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:07<00:00, 658.53it/s]

count     win    lose    draw
o        2993    1421     586
x        1450    2917     633
total    4443    4338    1219

ratio     win    lose    draw
o       59.9%   28.4%   11.7%
x       29.0%   58.3%   12.7%
total   44.4%   43.4%   12.2%



[('count',
  [{'win': 2993, 'lose': 1421, 'draw': 586},
   {'win': 1450, 'lose': 2917, 'draw': 633},
   {'win': 4443, 'lose': 4338, 'draw': 1219}],
  '7d'),
 ('ratio',
  [{'win': 0.5986, 'lose': 0.2842, 'draw': 0.1172},
   {'win': 0.29, 'lose': 0.5834, 'draw': 0.1266},
   {'win': 0.4443, 'lose': 0.4338, 'draw': 0.1219}],
  '7.1%')]